In [1]:
import gpytorch
import torch
import numpy as np
import metrics

In [2]:
train_x = torch.Tensor([0, 0])
train_y = torch.Tensor([0.1, 0.1])

In [3]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [4]:
adam = torch.optim.Adam(model.parameters(), 0.1)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
for i in range(50):
    output = model(train_x)
    loss = -mll(output, train_x)
    loss.backward() 
    adam.step()


In [5]:
output = model(train_x)
loss = -mll(output, train_x)
metrics.calculate_laplace(model, -loss, with_prior=False)

(tensor([[-8.0063]], dtype=torch.float64, grad_fn=<AddBackward0>),
 {'neg MLL': tensor(-1.7903, grad_fn=<NegBackward0>),
  'punish term': tensor([[-9.7965]], dtype=torch.float64),
  'punish without replacement': tensor([[-0.9787]], dtype=torch.float64),
  'num_replaced': tensor(3),
  'parameter list': ['likelihood.noise_covar.raw_noise',
   'covar_module.raw_outputscale',
   'covar_module.base_kernel.raw_lengthscale'],
  'parameter values': tensor([[-5.9927],
          [-5.9931],
          [ 0.0000]]),
  'corrected Hessian': tensor([[ 0.2775, -0.0400,  0.0000],
          [-0.0400,  0.2716,  0.0000],
          [ 0.0000,  0.0000,  0.3409]], dtype=torch.float64),
  'diag(constructed eigvals)': tensor([0.3409, 0.2344, 0.3146], dtype=torch.float64),
  'original symmetrized Hessian': tensor([[ 0.0639, -0.0540,  0.0000],
          [-0.0540,  0.0559,  0.0000],
          [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64),
  'prior mean': tensor([[-3.5164],
          [-1.6253],
          [-0.212

In [24]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    output = model(train_x)
    observed_pred = likelihood(output)
    

/home/besginow/anaconda3/envs/sage/lib/python3.10/site-packages/gpytorch/models/exact_gp.py:274: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


In [25]:
observed_pred.covariance_matrix

tensor([[0.0040, 0.0013],
        [0.0013, 0.0040]])

In [26]:
observed_pred.loc

tensor([0.0999, 0.0999])